In this notebook we suggest an approach to estimate future water levels of multiple waterbodies in northern and central Italy. Nine different waterbodies of 4 distinct types (acquifers, water springs, rivers & lakes) are studies, each with different target variables. Instead of aiming for the lowest possible error black box models, we aim to generate interpretable models that do not only give usable estimations, but also allow domain experts to interact with the models. External weather data is collected surrounding the waterbody locations, and we use simple dimension reduction techniques to engineer easy-to-interpret features. Those features are then used as inputs for rolling Gradient Boosted Tree (GBM) models, which fit the natural processes of water bodies very well (@thomas: referentie). We aim for this notebook to be concise and to the point and will therefor not extensively analyse each of the 20 target measurements or all of the input parameters, but leave the reader with documented code to further dive into points of his or her interest.

## Executive summary
